# 라이브러리 설정

In [125]:
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score

In [126]:
pm = pd.read_csv('data/pm_cls.csv') #전처리가 완료된 csv파일 불러오기
pm.reset_index(drop=True,inplace=True) #인덱스 초기화
#pm.drop('Unnamed: 0',axis=1,inplace=True) #필요없는 컬럼 제거
pm.head() #상위 5개의 csv 관찰

,제품 코드,제품명,원재료명,함량(%),분류
0,MI-151007-004,칠리소스,토마토페이스트,38.174524,소스
1,MI-151007-004,칠리소스,양파,13.159478,소스
2,MI-151007-004,칠리소스,양송이,11.574724,소스
3,MI-151007-004,칠리소스,향신지미베이스,7.582748,소스
4,MI-151007-004,칠리소스,베이컨,5.917753,소스


In [127]:
! pip3 install --target=$pg_path gensim==3.8.0 #gensim v3.8.0 이상에서 해야 로딩을 할 수 있음


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [128]:
pm.drop('제품 코드',axis=1,inplace=True)

# 정답 데이터셋 컬럼 통합하기

#### 정답 데이터셋에 대한 데이터프레임 만들기

In [129]:
pulmu = pm.copy()
pulmu

,제품명,원재료명,함량(%),분류
0,칠리소스,토마토페이스트,38.174524,소스
1,칠리소스,양파,13.159478,소스
2,칠리소스,양송이,11.574724,소스
3,칠리소스,향신지미베이스,7.582748,소스
4,칠리소스,베이컨,5.917753,소스
...,...,...,...,...
14900,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,멸치가루,0.010000,소스
14901,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,이소말토올리고당,1.200000,소스
14902,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,혼합제제,2.000000,소스
14903,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,우리밀짜장가루,3.300000,소스


#### FastText Training

In [130]:
import gensim
from gensim import models
from gensim.models import fasttext
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec, KeyedVectors

In [131]:
#solution1
X_train = list(pulmu['제품명'].unique())

In [132]:
#solution2
X_train1 = list(pulmu['제품명'].unique())
X_train2 = list(pulmu['원재료명'].unique())
X_train = X_train1 + X_train2

In [133]:
%%time
ft_model = fasttext.load_facebook_model('data/wiki.ko.bin') #FastText 불러오기
ft_model.build_vocab(sentences=X_train, update=True)
ft_model.train(X_train, total_examples=len(X_train),epochs=10)

CPU times: user 1min 26s, sys: 36.5 s, total: 2min 3s
Wall time: 2min 3s


데이터 프레임 조작

In [134]:
total = ['0']*len(pulmu)

for i in range(len(pm)):
    total[i] = pulmu['원재료명'][i] + ' ' + str(round(pulmu['함량(%)'][i],2))

In [135]:
pulmu['계'] = total

In [136]:
pulmu['함량(%)']=1

In [137]:
pulmu.drop('원재료명',axis=1,inplace=True)
pulmu

,제품명,함량(%),분류,계
0,칠리소스,1,소스,토마토페이스트 38.17
1,칠리소스,1,소스,양파 13.16
2,칠리소스,1,소스,양송이 11.57
3,칠리소스,1,소스,향신지미베이스 7.58
4,칠리소스,1,소스,베이컨 5.92
...,...,...,...,...
14900,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,1,소스,멸치가루 0.01
14901,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,1,소스,이소말토올리고당 1.2
14902,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,1,소스,혼합제제 2.0
14903,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,1,소스,우리밀짜장가루 3.3


원재료명 + 함량을 원재료명에 한번에 표현.

In [138]:
pulmu.columns = ['제품명','함량(%)','분류','원재료명']

In [139]:
pulmu = pulmu[['제품명','원재료명','함량(%)','분류']]
pulmu

,제품명,원재료명,함량(%),분류
0,칠리소스,토마토페이스트 38.17,1,소스
1,칠리소스,양파 13.16,1,소스
2,칠리소스,양송이 11.57,1,소스
3,칠리소스,향신지미베이스 7.58,1,소스
4,칠리소스,베이컨 5.92,1,소스
...,...,...,...,...
14900,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,멸치가루 0.01,1,소스
14901,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,이소말토올리고당 1.2,1,소스
14902,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,혼합제제 2.0,1,소스
14903,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,우리밀짜장가루 3.3,1,소스


# FastText + Node2Vec

In [140]:
pip install node2vec #Node2Vec 설치


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [141]:
pip install --upgrade scipy networkx


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [142]:
import networkx as nx
from node2vec import Node2Vec

#### Node2Vec 모델 훈련

첫번째 model은 함량 정보를 담고있는 제품-원재료명 훈련 모델

In [143]:
pulmu = pulmu.copy()
G = nx.Graph()

#product = pulmu['제품명'].unique().tolist()

# # edge
for i in range(len(pulmu)):
    G.add_edge(pulmu['제품명'][i],pulmu['원재료명'][i],relation='ingredients')


# 노드 임베딩
node2vec = Node2Vec(G,dimensions=20,walk_length=16,num_walks=100)
model = node2vec.fit(window=10,min_count=1)

Computing transition probabilities: 100%|█| 7894/7894 [00:00<0
Generating walks (CPU: 1): 100%|█| 100/100 [05:55<00:00,  3.55


두번째 model1은 함량 정보를 담지않은 제품-원재료명 훈련 모델

In [144]:
G1 = nx.Graph()

#product = pulmu['제품명'].unique().tolist()

# # edge
for i in range(len(pm)):
    G1.add_edge(pm['제품명'][i],pm['원재료명'][i],relation='ingredients')
#ingredient = pulmu['원재료명'].unique().tolist()
#product = pulmu['제품명'].unique().tolist()

# 노드 임베딩
node2vec = Node2Vec(G1,dimensions=20,walk_length=16,num_walks=100)
model1 = node2vec.fit(window=10,min_count=1)

Computing transition probabilities: 100%|█| 2343/2343 [00:03<0
Generating walks (CPU: 1): 100%|█| 100/100 [02:08<00:00,  1.29


# Product's ingredient node

첫번째 섹션

입력값과 유사한 단어(제품) top3를 찾고, 관련 ingredient node를 뽑는 작업을 합니다.

#### 입력 제품명과 가장 유사한 단어

In [151]:
import sys
# Product
def rank_list(name1,name2):
    rank = {}
    product_list = []
    clf = pulmu[pulmu['분류']==name2]
    X_train = list(clf['제품명'].unique())
    for i in range(len(X_train)):
        sim = ft_model.wv.similarity(name1,X_train[i])
        rank[X_train[i]]=sim
    ranked_list = sorted(rank.items(), key=lambda x: x[1],reverse=True)
    ranked_list = ranked_list[:5]
    ranked_list
    for i in range(5):
        product_list.append(ranked_list[i][0])
    return product_list

# ingredient
def node2ingredient(input_data):
    data= model.wv.most_similar(input_data,topn=100)

    answer_ingredient_list =[]

    for i in range(len(data)):
        for j in range(len(ingredient)):
            if(data[i][0]==ingredient[j]):
                answer_ingredient_list.append(data[i])
    ingredient_node =[]
    ingredient_node_sim =[]

    for i in range(len(answer_ingredient_list)):
        ingredient_node.append(answer_ingredient_list[i][0])
        ingredient_node_sim.append(answer_ingredient_list[i][1])

    ingredient_node_df = pd.DataFrame(data=list(zip(ingredient_node,ingredient_node_sim)),columns=['원재료','유사도'])
    ind = ingredient_node_df[:20]
    return list(ind['원재료'])

# 함량 백분율 통합
def integrate(ingred_list):
    dic = {}
    for i in ingred_list:    
        idx = i.index(' ')
        dic[i[0:idx]]=float(i[idx:])
    return dic

In [600]:
ingredient = pulmu['원재료명'].unique().tolist()
product = pulmu['제품명'].unique().tolist()

print('입력 형태 : 제품명,분류명 [ 예시 : 흑임자,소스  ]')
input_data1,input_data2 = map(str,input('제품을 넣어주세요 :').split(',')) #이름과 분류를 입력.
similar_rank = rank_list(input_data1,input_data2) #입력한 이름과 비슷한 product를 추출
top1_similar,top2_similar,top3_similar = similar_rank[0],similar_rank[1],similar_rank[2]

##################################################
print('★ Top_1 similar product is %s' %top1_similar)
print('★ Top_2 similar product is %s' %top2_similar)
print('★ Top_3 similar product is %s' %top3_similar)
##################################################

ingred_list,ingred_list_2,ingred_list_3 = node2ingredient(top1_similar),node2ingredient(top2_similar),node2ingredient(top3_similar)

# print('***********************************************************' * 2)
# print("★ Top_1 similar product's ingredient is %s" %ingred_list)
# print('***********************************************************' * 2)

# print("★ Top_2 similar product's ingredient is %s" %ingred_list_2)
# print('***********************************************************' * 2)

# print("★ Top_3 similar product's ingredient is %s" %ingred_list_3)
# print('***********************************************************' * 2)

dic1,dic2,dic3 = integrate(ingred_list), integrate(ingred_list_2), integrate(ingred_list_3)

dic1 = sorted(dic1.items(), key = lambda x:x[1],reverse=True)
dic2 = sorted(dic2.items(), key = lambda x:x[1],reverse=True)
dic3 = sorted(dic3.items(), key = lambda x:x[1],reverse=True)

# for i in dic1:
#     print(i)
# print('***********************************************************' * 2)
# for i in dic2:
#     print(i)
# print('***********************************************************' * 2)
# for i in dic3:
#     print(i)

입력 형태 : 제품명,분류명 [ 예시 : 흑임자,소스  ]


제품을 넣어주세요 : 흑임자,소스


★ Top_1 similar product is 참깨&흑임자드레싱(SB)
★ Top_2 similar product is 참깨흑임자드레싱
★ Top_3 similar product is 참깨흑임자드레싱1


In [601]:
df1,df2,df3 = pd.DataFrame(dic1),pd.DataFrame(dic2),pd.DataFrame(dic3)
df4 = pd.concat([df1,df2,df3],axis=1)
df4.fillna(' ',inplace=True)
df4.columns = [['top1','top1','top2','top2','top3','top3'], ['원재료','함량','원재료','함량','원재료','함량']]

In [602]:
df4 #top1,top2,top3 제품이 가지고 있는 관계 재료 노드 (가시성을 위해서 밑의 결과로 표현)

top1             top2               top3       
         원재료     함량       원재료     함량         원재료     함량
0       마요네즈   36.0       정제수  34.26         정제수  87.70
1        정제수  34.26      마요네즈   33.5        마요네즈  23.95
2       기타과당   14.0      액상과당   14.0        기타과당  10.01
3       액상과당   14.0      기타과당   14.0        발효식초   2.72
4       발효식초    3.8      발효식초    5.2      볶음참깨분말   2.50
5    향긋한사과식초    3.8    볶음참깨분말    5.0       냉면베이스   1.50
6     볶음참깨분말    3.5   향긋한사과식초    3.8        양조간장   1.29
7      볶음참깨분    3.5     볶음참깨분    3.5       볶음검정깨   0.86
8       양조간장    1.8      양조간장    1.8        정제소금   0.84
9      볶음검정깨    1.2     볶음검정깨    1.2         참기름   0.61
10      정제소금   1.17      정제소금   1.17  쇠고기양지베이스-P   0.30
11       참기름   0.85       참기름   0.85     평양지미베이스   0.30
12     후추조미분   0.32     후추조미분   0.32     반응풍미베이스   0.10
13       잔탄검   0.09       잔탄검   0.09       흑후추분말   0.07
14    밀분해추출물   0.05  아미노베이스-P   0.05         잔탄검   0.06
15  아미노베이스-P   0.05    밀분해추출물   0.05      밀분해추출물   0.04
16                        구연산   0.03    아미노베이스-P   0.04
17                                           구연산   0.02
18                                          배농축액   0.02

# 입력값과 유사한 원재료명들의 Node2Vec 관찰

두번째 섹션에서는 입력값과 유사한 원재료명을 찾고(FastText),

위의 결과에서 나온 원재료명의 Node2Vec찾습니다 (Node2Vec).

dic1,dic2,dic3는 입력 제품명과 유사도가 비슷한 제품이 가지고 있는 노드 ingredient 입니다.

In [603]:
new_dict = dic1 + dic2 + dic3

#### (1) 위의 유사한 제품명들이 갖고있는 ingredient node를 데이터프레임화

In [604]:
combination_by_product_df = pd.DataFrame(new_dict)
combination_by_product_df #df4와 같은 결과지만, 다른 표현

,0,1
0,마요네즈,36.00
1,정제수,34.26
2,기타과당,14.00
3,액상과당,14.00
4,발효식초,3.80
5,향긋한사과식초,3.80
6,볶음참깨분말,3.50
7,볶음참깨분,3.50
8,양조간장,1.80
9,볶음검정깨,1.20


입력값과 유사도가 비슷한 원재료명 rank

In [605]:
rank = {}
ingredient_list = []
X_train = list(pm['원재료명'].unique())
for i in range(len(X_train)):
    sim = ft_model.wv.similarity(input_data1,X_train[i])
    rank[X_train[i]]=sim
ranked_list = sorted(rank.items(), key=lambda x: x[1],reverse=True)
ranked_list = ranked_list[:10]
ranked_list
for i in range(5):
    ingredient_list.append(ranked_list[i][0])
print(ranked_list)

[('볶음흑임자갈음', 0.49649042), ('배퓨레', 0.47473556), ('크러쉬드레드페퍼분말', 0.45854115), ('크러쉬드레드페퍼', 0.45316744), ('키위가당퓨레', 0.4479907), ('데어리스프레드', 0.44440705), ('막사롬셀렉트파우더', 0.4443309), ('크랜베리퓨레', 0.44197553), ('아빠짜장베이스', 0.43479842), ('세이버리메이트1호', 0.43437526)]


입력과 유사한 원재료명들의 key값 리스트

In [606]:
new_ranked_list = list(itertools.chain.from_iterable(ranked_list))
new_ranked_list = new_ranked_list[0::2]
new_ranked_list

['볶음흑임자갈음',
 '배퓨레',
 '크러쉬드레드페퍼분말',
 '크러쉬드레드페퍼',
 '키위가당퓨레',
 '데어리스프레드',
 '막사롬셀렉트파우더',
 '크랜베리퓨레',
 '아빠짜장베이스',
 '세이버리메이트1호']

위의 각 key에 대한 ingredient노드를 추출

##### ★ ingredient Rank3 (입력값과 유사도가 높은 원재료명 top3)

In [607]:
new_ranked_list[0:3]

['볶음흑임자갈음', '배퓨레', '크러쉬드레드페퍼분말']

In [608]:
ingredient = pm['원재료명'].unique().tolist()
product = pulmu['제품명'].unique().tolist()

def node2ingredient_no_ingred(input_data):
    data= model1.wv.most_similar(input_data,topn=100)

    answer_ingredient_list =[]

    for i in range(len(data)):
        for j in range(len(ingredient)):
            if(data[i][0]==ingredient[j]):
                answer_ingredient_list.append(data[i])
    ingredient_node =[]
    ingredient_node_sim =[]

    for i in range(len(answer_ingredient_list)):
        ingredient_node.append(answer_ingredient_list[i][0])
        ingredient_node_sim.append(answer_ingredient_list[i][1])

    ingredient_node_df = pd.DataFrame(data=list(zip(ingredient_node,ingredient_node_sim)),columns=['원재료','유사도'])
    ind = ingredient_node_df[:20]
    return list(ind['원재료'])

##### ★ ingredient Rank3's node ingredient (위의 top3 각각의 인접하는 재료 노드)

In [609]:
list_ = list(set(node2ingredient_no_ingred(new_ranked_list[0]) + node2ingredient_no_ingred(new_ranked_list[1]) + node2ingredient_no_ingred(new_ranked_list[2])))
print(list_)

['변성전분', "5'-리보뉴클레티드이나트륨", '정제염', '막국수비빔베이스', '고추장(고추장발효물)', '갈릭크러쉬', '셀러리', '조미페이스트1022', '6가지과채동치미농축액', '미강유', '크러쉬드토마토', '우마미조미액', '배농축과즙액', '볶음참깨', '바질', '향신조미분-에이치에프3', '와사비소스P', 'VertexIG20', '조개엑기스NM', '땅콩분태', '월계수잎분말', '바질분말', '효모추출물(이스트엑기스)', '청피망', '맑은옥수수유', '퓨리티씨에스씨', '함흥냉면지미베이스', '배농축액(신화무역)', '네오HP', '맛뜨락참기름', '오레가노', '향신조미분-에이치에프4', '와사비소스', '헬스리고(이소말토올리고당)', '배양분말', '냉동다진마늘', '가쓰오농축액P', '소프리토', '잘라페노페퍼', '나초잘라페노스', '다이스드토마토', '나쵸슬라이스할라페노페퍼스', '향긋한사과식초', '과당', '프리미엄고추장', '사과농축액', '조미액베이스S-99', '막국수베이스분말', '현미유', '월계수잎', '진한육수베이스분말', '이스트엑기스STV', '청양고추믹스', '고쿠미조미료', '후라이드갈릭후레이크', '바질리브즈', '양조간장', '볶음흑참깨', '조개엑기스']


#### (2) 추출된 노드들을 데이터프레임화

In [610]:
combination_by_ingredient_df=pd.DataFrame(list_)

#### (1)과 (2)를 병합

##### ★merge 결과

In [611]:
merge_df = pd.merge(combination_by_product_df,combination_by_ingredient_df)
merge_df

,0,1
0,향긋한사과식초,3.80
1,향긋한사과식초,3.80
2,양조간장,1.80
3,양조간장,1.80
4,양조간장,1.29


데이터프레임1의 TOP3 제품 각각의 상단에서 함량순3, 중단에서 함량순3, 하단에서 함량순3을 뽑기 위한 작업

### 섹션1의 결과를 가지고 combination을 합니다.

In [612]:
import itertools

_dic1 = list(itertools.chain.from_iterable(dic1))
_dic1 = _dic1[0::2]
_dic2 = list(itertools.chain.from_iterable(dic2))
_dic2 = _dic2[0::2]
_dic3 = list(itertools.chain.from_iterable(dic3))
_dic3 = _dic3[0::2]

In [613]:
result = ()
result = set(_dic1[0:int(len(_dic1)*0.33)])
result

{'기타과당', '마요네즈', '발효식초', '액상과당', '정제수'}

In [614]:
cnt = 0
start = int(len(_dic2)*0.33)
end = int(len(_dic2)*0.66)

for v2 in _dic2[start:end] :
    
    result.add(v2)
    cnt = len(result)
    
    if cnt == 6 :
        break
    
result

{'기타과당', '마요네즈', '발효식초', '볶음참깨분말', '액상과당', '정제수'}

In [615]:
cnt = 0
start = int(len(_dic2)*0.66)


for v3 in _dic3[start:] :
    result.add(v3)
    cnt = len(result)
    
    if cnt == 9 :
        break
    
combination = result

##### ▲top3 각각의 상,중,하 ingredient 추출 작업 끝

---------------------------------------------------------------------------------------------------

위의 결과에 해당하는 값들만 뽑음

##### ★ Combination 결과

In [616]:
combination

{'기타과당',
 '마요네즈',
 '반응풍미베이스',
 '발효식초',
 '볶음참깨분말',
 '액상과당',
 '정제수',
 '평양지미베이스',
 '흑후추분말'}

In [617]:
df10 = pd.DataFrame()

for i in combination:
    df10 = df10.append(combination_by_product_df[combination_by_product_df[0]==i])
#df10

#MERGE데이터프레임과 조합데이터프레임을 concat하고, 중복된 값은 하나만 남김(함량은 평균내고) ->final데이터프레임

★ result

In [618]:
final_df = pd.concat([merge_df, df10])

In [619]:
result = final_df.groupby(0)[1].mean()
result = pd.DataFrame(result)
result.reset_index(drop=False, inplace= True)
result.columns = ['원재료명','함량']

In [620]:
result

,원재료명,함량
0,기타과당,12.670000
1,마요네즈,31.150000
2,반응풍미베이스,0.100000
3,발효식초,3.906667
4,볶음참깨분말,3.666667
5,액상과당,14.000000
6,양조간장,1.630000
7,정제수,52.073333
8,평양지미베이스,0.300000
9,향긋한사과식초,3.800000


##### ★ 백분율 통합

In [621]:
f_df = result.copy()
p = 100 / f_df['함량'].sum(axis=0)
f_df['함량'] = f_df['함량'] * p
f_df

,원재료명,함량
0,기타과당,10.270197
1,마요네즈,25.249932
2,반응풍미베이스,0.081059
3,발효식초,3.166712
4,볶음참깨분말,2.972170
5,액상과당,11.348284
6,양조간장,1.321265
7,정제수,42.210213
8,평양지미베이스,0.243178
9,향긋한사과식초,3.080249


★ 함량 재조정 (ROOT)

In [622]:
import numpy as np

f_df = result.copy()
f_df['함량'] = np.sqrt(f_df['함량'])

p = 100 / f_df['함량'].sum(axis=0)
f_df['함량'] = f_df['함량'] * p
f_df

,원재료명,함량
0,기타과당,12.557955
1,마요네즈,19.690632
2,반응풍미베이스,1.115657
3,발효식초,6.973227
4,볶음참깨분말,6.755637
5,액상과당,13.200629
6,양조간장,4.504270
7,정제수,25.458822
8,평양지미베이스,1.932374
9,향긋한사과식초,6.877370
